# Indeed Scrapper

### Indeed Company Reviews Scrapper

The way the links work in this website makes it easier to scrape the data as it loops by 20 steps
so it's easy to set it up in a for range and counting 20 steps and there's no need to use for example clicks
and the data is already all visible in the html 
The link is also dynamic and allows to exchange the companies via the link so the company was made as variable


The website reviews are stored in a general container, if we try to scrape directly  the instead of going
inside of the container the we're getting empty values

In [1]:
#Import Necessary Librabries
import time
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
import random

In [10]:
#Adding the argument to make it not visible
options = Options()
options.add_argument('--headless')
#options.add_argument('--disable-gpu')  # this was necessary in windows/chrome
options.add_argument("--disable-blink-features") # avoid Robot detection
options.add_argument("--disable-blink-features=AutomationControlled")

In [11]:
#Open Chrome with Selenium
browser = webdriver.Chrome('E:\Documentos\chromedriver.exe', options = options)  # Optional argument, if not specified will search path.


# Open Firefox with Selenium (unmark to run with firefox)
#binary = FirefoxBinary('/Applications/Firefox.app/Contents/MacOS/firefox-bin')  # replace the Firexfox path with the one on your computer
#browser = webdriver.Firefox(firefox_binary=binary, options=options)
#browser = webdriver.Firefox(options=options)

In [4]:
#Creating a DataFrame to store the data
extracted_reviews = pd.DataFrame({'review title': pd.Series([], dtype='string'),
                                    'review':pd.Series([], dtype='string'),
                                    'pros':pd.Series([], dtype='string'),
                                    'cons':pd.Series([], dtype='string'),
                                    'date':pd.Series([], dtype='string'),
                                    'job':pd.Series([], dtype='string'),
                                    'location':pd.Series([], dtype='string'),
                                    'rating':pd.Series([], dtype='float')})

In [12]:
def Indeed_Scrapper(company,country,loops,extracted_reviews):
    print('Pulling Reviews for ',company)
    run = 0

    for i in range(0,loops*20,20):
        try:
            #The link select has all the countries and the language selected was english
            link = (f"https://www.indeed.com/cmp/{company}/reviews?fcountry={country}&start={i}&lang=en") 
            browser.get(link)
            time.sleep(5)
            html_source = browser.page_source
            browser.quit()
            soup = BeautifulSoup(html_source,"html.parser")
            #browser.quit()
        except:
            print("Link didn't work")
        
        try:
            # Reviews are divided by containers and not all of them have the same attributes for we're scrapping per container
            data = soup.find('div',{"id": 'cmp-container'})
            reviews = data.find_all(class_ = "cmp-Review-container")
            for x in reviews:
                #Pull the title
                title = x.find(attrs = {'class':'cmp-Review-title'})
                
                #Pull the review content
                review = x.find('div', {'class': 'cmp-Review-text'})
                
                #Not all reviews have the pros and cons (but if one exists both do)
                try:
                    pros = x.find(attrs = {'class':'cmp-ReviewProsCons-prosText'}).find(attrs = {'class': 'cmp-NewLineToBr-text'}).get_text()
                    cons = x.find(attrs = {'class':'cmp-ReviewProsCons-consText'}).find(attrs = {'class': 'cmp-NewLineToBr-text'}).get_text()
                except:
                    pros,cons = None,None
                    
                # Pull the Review Date
                rev_date = x.find(attrs = {'class':'cmp-Review-author'})
                
                # Pull the Job title(not always present)
                try:
                    job = x.find('a', {'class':'cmp-ReviewAuthor-link'}).text
                except:
                    job = None
                    
                #Pull the location
                location = x.find(attrs = {'class':'cmp-Review-author'})
                
                # Pull the rating given by the reviwer
                rating = x.find(attrs = {'class':'cmp-ReviewRating-text'})
                
                # Save the data to the DataFrame and make the necessary changes to the text
                extracted_reviews = extracted_reviews.append({'review title': title.text,
                                                          'review': review.text,
                                                          'pros': pros,
                                                          'cons': cons,
                                                          'date': rev_date.text.split("-")[-1],
                                                          'job': job,
                                                          'location': location.text.split("-")[-2],
                                                          'rating': rating.text
                                                          }, ignore_index=True)
                print(run)
                run += 1
                time.sleep(random.randint(1, 10))
        except:
            print("Something went wrong! Try another time..")
            return[]
    
    #Convert the reviews as float 
    extracted_reviews['rating']= [x.replace(",",".") for x in extracted_reviews['rating']]   
   
    #Save to excel file
    extracted_reviews.to_excel("Mcdonald's Reviews.xlsx")
    
    print("Reviews Saved!")

In [13]:
Indeed_Scrapper("Mcdonald's","GB",2,extracted_reviews)

Pulling Reviews for  Mcdonald's
Link didn't work
